# 社会互动的重要性

In [87]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import random
from collections import Counter

In [2]:
df=pd.read_csv('D:/mhx/daima/pachong/zhibopachong/dataprocess/BGxM3akaUZYjTlaZ7bZi9th9e34npLl6.csv')
#df=pd.read_csv('D:/social network1/daima/pachong/zhibopachong/dataprocess/BGxM3akaUZYjTlaZ7bZi9th9e34npLl6.csv')
df.head(5)

avg_residence_time  crawl_time  follower_count  follower_count_incr  \
0                   0  1682254298            3217                    0   
1                   0  1682254378            3217                    0   
2                  30  1682254392            3217                    0   
3                  28  1682254991            3218                    1   
4                  41  1682256196            3218                    0   

   gift_uv_count  increment_follow_count  increment_follow_count_incr  \
0              0                       0                            0   
1              0                       0                            0   
2              0                       0                            0   
3              0                       1                            1   
4              0                       1                            0   

   leave_user  like_count  like_count_incr  ...  room_ticket_count_incr  \
0           0           0                0  ...                       0   
1           0           0                0  ...                       0   
2           0           0                0  ...                       0   
3           2           0                0  ...                       0   
4           1           0                0  ...                       0   

   status  status_v2   time_node  total_user  total_user_incr  user_count  \
0       0          0  1682254260           0                0           0   
1       2          0  1682254320           0                0           0   
2       2          0  1682254380           1                1           1   
3       2          0  1682254980          21               20           1   
4       2          0  1682256180          49               28           1   

   user_count_incr  user_peak  watch_cnt  
0                0          0          0  
1                0          0          0  
2                1          1          1  
3                0          1         21  
4                0          1         49  

[5 rows x 24 columns]

In [36]:
def add_remove(G,t,to_remove):
    # 添加新节点和新边
    new_node =list(range(G.number_of_nodes(), G.number_of_nodes()+df['real_time_user'][t]))
    for i in new_node:
        G.add_node(i)
        new_node_idx = G.number_of_nodes() - 1
        for j in range(new_node_idx):
            G.add_edge(new_node_idx, j)
    # 随机选择要退出的节点
    # 设置随机种子
    random.seed(42)
    to_remove.extend(random.sample(range ( 1, G.number_of_nodes()), df['leave_user'][t]))
    # 从图中删除leave_user其相连的边
    for node in to_remove:
        neighbors = list(G.neighbors(node))
        for neighbor in neighbors:
            G.remove_edge(node, neighbor)
    return G,to_remove,new_node

In [37]:
def generate_snapshots(num_snapshots):
    snapshots = []
    to_remove = []
    G = nx.complete_graph(1)
    for t in range(0, num_snapshots):
        G, to_remove, new_nodes = add_remove(G, t, to_remove)
        snapshots.append(G.copy())
    return snapshots
# 生成网络快照
snapshots = generate_snapshots(df.shape[0])

In [88]:
edge_list=[]
node_list=[]
node_dict={}#空字典用于储存节点第一次出现的时间
#提取节点和边的事件
for i,snapshot in enumerate(snapshots):
    edges=[(u,v,i) for u,v in snapshot.edges()]
    nodes=list(snapshot.nodes())
    edge_list.extend(edges)
    for node in nodes:
        if node not in node_dict:
            node_dict[node]=i
df_edges=pd.DataFrame(edge_list,columns=['source','target','online_time'])
df_nodes=pd.DataFrame(node_dict.items(),columns=['node','online_time'])

In [74]:
df_edges.head(50)

source  target  online_time
0        0       1            2
1        0       2            3
2        0       2            4
3        0       4            4
4        2       4            4
5        0       2            5
6        0       4            5
7        0       5            5
8        0       7            5
9        2       4            5
10       2       5            5
11       2       7            5
12       4       5            5
13       4       7            5
14       5       7            5
15       0       4            6
16       0       5            6
17       0       7            6
18       0       8            6
19       4       5            6
20       4       7            6
21       4       8            6
22       5       7            6
23       5       8            6
24       7       8            6
25       0       4            7
26       0       5            7
27       0       7            7
28       0       8            7
29       0       9            7
30       0      10            7
31       4       5            7
32       4       7            7
33       4       8            7
34       4       9            7
35       4      10            7
36       5       7            7
37       5       8            7
38       5       9            7
39       5      10            7
40       7       8            7
41       7       9            7
42       7      10            7
43       8       9            7
44       8      10            7
45       9      10            7
46       0       7            8
47       0       8            8
48       0       9            8
49       0      12            8

In [49]:
df_nodes

node  online_time
0      0            0
1      1            2
2      2            3
3      3            3
4      4            4
5      5            5
6      6            5
7      7            5
8      8            6
9      9            7
10    10            7
11    11            8
12    12            8
13    13            8
14    14            8
15    15            9
16    16           10
17    17           10
18    18           12
19    19           13
20    20           13
21    21           14
22    22           14
23    23           15
24    24           16
25    25           17
26    26           19
27    27           24
28    28           27
29    29           31
30    30           34
31    31           37
32    32           40
33    33           41
34    34           43
35    35           46
36    36           48
37    37           50
38    38           50
39    39           53
40    40           56
41    41           59
42    42           59
43    43           59
44    44           62
45    45           63
46    46           65
47    47           66
48    48           66
49    49           69
50    50           72
51    51           73
52    52           75
53    53           76
54    54           79
55    55           81
56    56           82
57    57           83

# 动态特征

节点的时间特征

In [38]:
num_snapshots=df.shape[0]
#读取最后一张快照的节点数
last_snapshot=snapshots[-1]
max_nodes=last_snapshot.number_of_nodes()#58

In [41]:
def add_remove_time(G,t,to_remove,earliest_disappear_time):
    # 添加新节点和新边
    new_node =list(range(G.number_of_nodes(), G.number_of_nodes()+df['real_time_user'][t]))
    for i in new_node:
        G.add_node(i)
        new_node_idx = G.number_of_nodes() - 1
        for j in range(new_node_idx):
            G.add_edge(new_node_idx, j)
    # 随机选择要退出的节点
    # 设置随机种子
    random.seed(42)
    to_remove.extend(random.sample(range ( 1, G.number_of_nodes()), df['leave_user'][t]))

    #记录节点下线的时间戳
    for node in to_remove:
        if earliest_disappear_time[node] is None or t<earliest_disappear_time[node]:
            earliest_disappear_time[node]=t
    
    # 从图中删除leave_user其相连的边
    for node in to_remove:
        neighbors = list(G.neighbors(node))
        for neighbor in neighbors:
            G.remove_edge(node, neighbor)
    return G,to_remove,new_node,earliest_disappear_time

In [42]:
snapshots_time = []
to_remove = []
earliest_disappear_time=[None]*max_nodes
G = nx.complete_graph(1)
for t in range(0, num_snapshots):
    G, to_remove, new_nodes, earliest_disappear_time= add_remove_time(G, t, to_remove,earliest_disappear_time)
    snapshots_time.append(G.copy())
earliest_disappear_time

[None,
 3,
 6,
 3,
 8,
 8,
 5,
 63,
 14,
 14,
 8,
 8,
 10,
 None,
 None,
 63,
 63,
 None,
 14,
 None,
 None,
 14,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 63,
 None,
 None,
 None,
 None,
 None,
 60,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

将最后一张快照的在线节点的下线时间定为最后一个时间戳

In [43]:
last_snapshot_idx=df.shape[0]-1
for i,timestamp in enumerate(earliest_disappear_time):
    if timestamp is None:
        earliest_disappear_time[i]=last_snapshot_idx
earliest_disappear_time

[88,
 3,
 6,
 3,
 8,
 8,
 5,
 63,
 14,
 14,
 8,
 8,
 10,
 88,
 88,
 63,
 63,
 88,
 14,
 88,
 88,
 14,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 63,
 88,
 88,
 88,
 88,
 88,
 60,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88,
 88]

In [51]:
df_node_disappear=pd.DataFrame({'node':range(len(earliest_disappear_time)),'offline_time':earliest_disappear_time})
df_nodes=pd.merge(df_nodes,df_node_disappear,on='node',how='left')
df_nodes

node  online_time  offline_time
0      0            0            88
1      1            2             3
2      2            3             6
3      3            3             3
4      4            4             8
5      5            5             8
6      6            5             5
7      7            5            63
8      8            6            14
9      9            7            14
10    10            7             8
11    11            8             8
12    12            8            10
13    13            8            88
14    14            8            88
15    15            9            63
16    16           10            63
17    17           10            88
18    18           12            14
19    19           13            88
20    20           13            88
21    21           14            14
22    22           14            88
23    23           15            88
24    24           16            88
25    25           17            88
26    26           19            88
27    27           24            88
28    28           27            88
29    29           31            88
30    30           34            88
31    31           37            88
32    32           40            88
33    33           41            88
34    34           43            88
35    35           46            63
36    36           48            88
37    37           50            88
38    38           50            88
39    39           53            88
40    40           56            88
41    41           59            60
42    42           59            88
43    43           59            88
44    44           62            88
45    45           63            88
46    46           65            88
47    47           66            88
48    48           66            88
49    49           69            88
50    50           72            88
51    51           73            88
52    52           75            88
53    53           76            88
54    54           79            88
55    55           81            88
56    56           82            88
57    57           83            88

边的时间特征

In [54]:
duplicate_edges=df_edges[df_edges.duplicated(subset=['source','target'],keep=False)]
duplicate_edges

source  target  online
1           0       2       3
2           0       2       4
3           0       4       4
4           2       4       4
5           0       2       5
...       ...     ...     ...
26184      54      56      88
26185      54      57      88
26186      55      56      88
26187      55      57      88
26188      56      57      88

[26137 rows x 3 columns]

In [89]:
edge_disappear_time=[]
for index,row in df_edges.iterrows():
    source=row['source']
    target=row['target']
    edge_appear_time=row['online_time']
    #在数据框中查询边出现后的所有记录
    later_records=df_edges[(df_edges['source']==source)&(df_edges['target']==target)&(df_edges['online_time']>edge_appear_time)]
    #若存在后续记录，取出最大的时间戳作为边的消失时间戳
    if not later_records.empty:
        disappear_time=later_records['online_time'].max()+1#下一时间戳下线
    else:
        disappear_time=edge_appear_time+1
    edge_disappear_time.append(disappear_time)
df_edges['offline_time']=edge_disappear_time
#删除重复边，只保留出现时间最小的行
df_edges_time=df_edges[~df_edges.duplicated(subset=['source','target'],keep='first')]
df_edges_time

source  target  online_time  offline_time
0           0       1            2             3
1           0       2            3             6
3           0       4            4             8
4           2       4            4             6
7           0       5            5             8
...       ...     ...          ...           ...
22278      52      57           83            89
22282      53      57           83            89
22285      54      57           83            89
22287      55      57           83            89
22288      56      57           83            89

[981 rows x 4 columns]

In [78]:
duplicate_edges1=df_edges[df_edges.duplicated(subset=['source','target'],keep=False)]
duplicate_edges1

Empty DataFrame
Columns: [source, target, online_time, offline_time]
Index: []

# 结构特征

聚合网络

In [80]:
def convert_to_max_nodes(snapshots,max_nodes):
    for snapshot in snapshots:
        #获取当前快照的节点数
        num_nodes=snapshot.number_of_nodes()
        if num_nodes<max_nodes:
            #添加额外的节点使其达到最大节点数
            extra_nodes=max_nodes-num_nodes
            new_nodes=list(range(num_nodes,num_nodes+extra_nodes))
            snapshot.add_nodes_from(new_nodes)
    return snapshots
#读取最后一张快照的节点数
last_snapshot=snapshots[-1]
max_nodes=last_snapshot.number_of_nodes()#58
convert_snapshots=convert_to_max_nodes(snapshots,max_nodes)

In [81]:
AL_AGG = nx.Graph()
al_agg = np.zeros((max_nodes, max_nodes))
#al_agg = np.zeros((nodes, nodes))
for go in convert_snapshots:
    go = nx.convert_node_labels_to_integers(go, first_label=0, ordering="sorted")
    AL_AGG = nx.compose(AL_AGG, go)
    al_agg += nx.to_numpy_matrix(go)

节点的结构特征

In [92]:
#度
degree=dict(AL_AGG.degree())
degree

{0: 53,
 1: 1,
 2: 4,
 3: 0,
 4: 7,
 5: 7,
 6: 0,
 7: 39,
 8: 15,
 9: 15,
 10: 6,
 11: 0,
 12: 7,
 13: 48,
 14: 48,
 15: 35,
 16: 34,
 17: 47,
 18: 11,
 19: 47,
 20: 47,
 21: 0,
 22: 44,
 23: 44,
 24: 44,
 25: 44,
 26: 44,
 27: 44,
 28: 44,
 29: 44,
 30: 44,
 31: 44,
 32: 44,
 33: 44,
 34: 44,
 35: 31,
 36: 44,
 37: 44,
 38: 44,
 39: 44,
 40: 44,
 41: 30,
 42: 44,
 43: 44,
 44: 43,
 45: 39,
 46: 39,
 47: 39,
 48: 39,
 49: 39,
 50: 39,
 51: 39,
 52: 39,
 53: 39,
 54: 39,
 55: 39,
 56: 39,
 57: 39}

In [83]:
#聚类系数
clustering_coefficient=nx.clustering(AL_AGG)
clustering_coefficient

{0: 0.6734397677793904,
 1: 0,
 2: 1.0,
 3: 0,
 4: 0.8571428571428571,
 5: 0.8571428571428571,
 6: 0,
 7: 0.6909581646423751,
 8: 0.6952380952380952,
 9: 0.6952380952380952,
 10: 1.0,
 11: 0,
 12: 1.0,
 13: 0.8093971631205674,
 14: 0.8093971631205674,
 15: 0.8352941176470589,
 16: 0.875222816399287,
 17: 0.8390379278445883,
 18: 1.0,
 19: 0.8390379278445883,
 20: 0.8390379278445883,
 21: 0,
 22: 0.9302325581395349,
 23: 0.9302325581395349,
 24: 0.9302325581395349,
 25: 0.9302325581395349,
 26: 0.9302325581395349,
 27: 0.9302325581395349,
 28: 0.9302325581395349,
 29: 0.9302325581395349,
 30: 0.9302325581395349,
 31: 0.9302325581395349,
 32: 0.9302325581395349,
 33: 0.9302325581395349,
 34: 0.9302325581395349,
 35: 0.9978494623655914,
 36: 0.9302325581395349,
 37: 0.9302325581395349,
 38: 0.9302325581395349,
 39: 0.9302325581395349,
 40: 0.9302325581395349,
 41: 1.0,
 42: 0.9302325581395349,
 43: 0.9302325581395349,
 44: 0.9424141749723145,
 45: 1.0,
 46: 1.0,
 47: 1.0,
 48: 1.0,
 49: 1

In [84]:
#介数中心性
centrality=nx.betweenness_centrality(AL_AGG)
centrality

{0: 0.11796588030923875,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.00046992481203007516,
 5: 0.00046992481203007516,
 6: 0.0,
 7: 0.044495166487647675,
 8: 0.004676870748299319,
 9: 0.004676870748299319,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.016462120910742468,
 14: 0.016462120910742468,
 15: 0.008154314357321885,
 16: 0.0048245614035087765,
 17: 0.010417246820755581,
 18: 0.0,
 19: 0.010417246820755581,
 20: 0.010417246820755581,
 21: 0.0,
 22: 0.0015408892601875052,
 23: 0.0015408892601875052,
 24: 0.0015408892601875052,
 25: 0.0015408892601875052,
 26: 0.0015408892601875052,
 27: 0.0015408892601875052,
 28: 0.0015408892601875052,
 29: 0.0015408892601875052,
 30: 0.0015408892601875052,
 31: 0.0015408892601875052,
 32: 0.0015408892601875052,
 33: 0.0015408892601875052,
 34: 0.0015408892601875052,
 35: 2.0885547201336673e-05,
 36: 0.0015408892601875052,
 37: 0.0015408892601875052,
 38: 0.0015408892601875052,
 39: 0.0015408892601875052,
 40: 0.0015408892601875052,
 41: 0.0,
 42: 0.001540889260187

In [ ]:
#同质性
assortactivity=nx.attribute_assortactivity_coefficient(AL_AGG,'attribute_name')
assortactivity

In [ ]:
#影响力传播
#著名算法：独立级联模型和线性阈值模型

In [98]:
df_node_feature=pd.DataFrame(columns=['node','degree','clustering_coefficient','centrality'])
for node in AL_AGG.nodes():
    row={'node':int(node),'degree':degree[node],'clustering_coefficient':clustering_coefficient[node],'centrality':centrality[node]}
    df_node_feature=df_node_feature.append(row,ignore_index=True)
df_node_feature

node  degree  clustering_coefficient  centrality
0    0.0    53.0                0.673440    0.117966
1    1.0     1.0                0.000000    0.000000
2    2.0     4.0                1.000000    0.000000
3    3.0     0.0                0.000000    0.000000
4    4.0     7.0                0.857143    0.000470
5    5.0     7.0                0.857143    0.000470
6    6.0     0.0                0.000000    0.000000
7    7.0    39.0                0.690958    0.044495
8    8.0    15.0                0.695238    0.004677
9    9.0    15.0                0.695238    0.004677
10  10.0     6.0                1.000000    0.000000
11  11.0     0.0                0.000000    0.000000
12  12.0     7.0                1.000000    0.000000
13  13.0    48.0                0.809397    0.016462
14  14.0    48.0                0.809397    0.016462
15  15.0    35.0                0.835294    0.008154
16  16.0    34.0                0.875223    0.004825
17  17.0    47.0                0.839038    0.010417
18  18.0    11.0                1.000000    0.000000
19  19.0    47.0                0.839038    0.010417
20  20.0    47.0                0.839038    0.010417
21  21.0     0.0                0.000000    0.000000
22  22.0    44.0                0.930233    0.001541
23  23.0    44.0                0.930233    0.001541
24  24.0    44.0                0.930233    0.001541
25  25.0    44.0                0.930233    0.001541
26  26.0    44.0                0.930233    0.001541
27  27.0    44.0                0.930233    0.001541
28  28.0    44.0                0.930233    0.001541
29  29.0    44.0                0.930233    0.001541
30  30.0    44.0                0.930233    0.001541
31  31.0    44.0                0.930233    0.001541
32  32.0    44.0                0.930233    0.001541
33  33.0    44.0                0.930233    0.001541
34  34.0    44.0                0.930233    0.001541
35  35.0    31.0                0.997849    0.000021
36  36.0    44.0                0.930233    0.001541
37  37.0    44.0                0.930233    0.001541
38  38.0    44.0                0.930233    0.001541
39  39.0    44.0                0.930233    0.001541
40  40.0    44.0                0.930233    0.001541
41  41.0    30.0                1.000000    0.000000
42  42.0    44.0                0.930233    0.001541
43  43.0    44.0                0.930233    0.001541
44  44.0    43.0                0.942414    0.001207
45  45.0    39.0                1.000000    0.000000
46  46.0    39.0                1.000000    0.000000
47  47.0    39.0                1.000000    0.000000
48  48.0    39.0                1.000000    0.000000
49  49.0    39.0                1.000000    0.000000
50  50.0    39.0                1.000000    0.000000
51  51.0    39.0                1.000000    0.000000
52  52.0    39.0                1.000000    0.000000
53  53.0    39.0                1.000000    0.000000
54  54.0    39.0                1.000000    0.000000
55  55.0    39.0                1.000000    0.000000
56  56.0    39.0                1.000000    0.000000
57  57.0    39.0                1.000000    0.000000

边的结构特征

In [101]:
#边的介数中心性
edge_betweenness=nx.edge_betweenness_centrality(AL_AGG)
edge_betweenness

{(0, 1): 0.032062915910465825,
 (0, 2): 0.019207501512401696,
 (0, 4): 0.01739261947973382,
 (0, 5): 0.01739261947973382,
 (0, 7): 0.0015011987183796021,
 (0, 8): 0.004217920279626264,
 (0, 9): 0.004217920279626264,
 (0, 10): 0.01754385964912281,
 (0, 12): 0.00780255235790625,
 (0, 13): 0.0019661222020568663,
 (0, 14): 0.0019661222020568663,
 (0, 15): 0.002257399565324551,
 (0, 16): 0.0023438225192611165,
 (0, 17): 0.002052545155993432,
 (0, 18): 0.004909303911118788,
 (0, 19): 0.002052545155993432,
 (0, 20): 0.002052545155993432,
 (0, 22): 0.002742488404920347,
 (0, 23): 0.002742488404920347,
 (0, 24): 0.002742488404920347,
 (0, 25): 0.002742488404920347,
 (0, 26): 0.002742488404920347,
 (0, 27): 0.002742488404920347,
 (0, 28): 0.002742488404920347,
 (0, 29): 0.002742488404920347,
 (0, 30): 0.002742488404920347,
 (0, 31): 0.002742488404920347,
 (0, 32): 0.002742488404920347,
 (0, 33): 0.002742488404920347,
 (0, 34): 0.002742488404920347,
 (0, 35): 0.003033765768188028,
 (0, 36): 0.002

In [102]:
#权重
edge_frequency={}
for snapshot in snapshots:
    for edge in snapshot.edges():
        if edge in edge_frequency:
            edge_frequency[edge]+=1
        else:
            edge_frequency[edge]=0
edge_frequency           

{(0, 1): 0,
 (0, 2): 2,
 (0, 4): 3,
 (2, 4): 1,
 (0, 5): 2,
 (0, 7): 57,
 (2, 5): 0,
 (2, 7): 0,
 (4, 5): 2,
 (4, 7): 2,
 (5, 7): 2,
 (0, 8): 7,
 (4, 8): 1,
 (5, 8): 1,
 (7, 8): 7,
 (0, 9): 6,
 (0, 10): 0,
 (4, 9): 0,
 (4, 10): 0,
 (5, 9): 0,
 (5, 10): 0,
 (7, 9): 6,
 (7, 10): 0,
 (8, 9): 6,
 (8, 10): 0,
 (9, 10): 0,
 (0, 12): 1,
 (0, 13): 80,
 (0, 14): 80,
 (7, 12): 1,
 (7, 13): 54,
 (7, 14): 54,
 (8, 12): 1,
 (8, 13): 5,
 (8, 14): 5,
 (9, 12): 1,
 (9, 13): 5,
 (9, 14): 5,
 (12, 13): 1,
 (12, 14): 1,
 (13, 14): 80,
 (0, 15): 53,
 (7, 15): 53,
 (8, 15): 4,
 (9, 15): 4,
 (12, 15): 0,
 (13, 15): 53,
 (14, 15): 53,
 (0, 16): 52,
 (0, 17): 78,
 (7, 16): 52,
 (7, 17): 52,
 (8, 16): 3,
 (8, 17): 3,
 (9, 16): 3,
 (9, 17): 3,
 (13, 16): 52,
 (13, 17): 78,
 (14, 16): 52,
 (14, 17): 78,
 (15, 16): 52,
 (15, 17): 52,
 (16, 17): 52,
 (0, 18): 1,
 (7, 18): 1,
 (8, 18): 1,
 (9, 18): 1,
 (13, 18): 1,
 (14, 18): 1,
 (15, 18): 1,
 (16, 18): 1,
 (17, 18): 1,
 (0, 19): 75,
 (0, 20): 75,
 (7, 19): 49,
 (7

In [105]:
df_link_feature=pd.DataFrame(columns=['edge','edge_betweenness','frequency'])
for edge,frequency in edge_frequency.items():
    row={'edge':edge,'edge_betweenness':edge_betweenness[edge],'frequency':edge_frequency[edge] }
    df_link_feature=df_link_feature.append(row,ignore_index=True)
df_link_feature

edge  edge_betweenness frequency
0      (0, 1)          0.032063         0
1      (0, 2)          0.019208         2
2      (0, 4)          0.017393         3
3      (2, 4)          0.001059         1
4      (0, 5)          0.017393         2
..        ...               ...       ...
976  (52, 57)          0.000605         5
977  (53, 57)          0.000605         5
978  (54, 57)          0.000605         5
979  (55, 57)          0.000605         5
980  (56, 57)          0.000605         5

[981 rows x 3 columns]

# 基本属性